<a href="https://colab.research.google.com/github/ubsuny/PHY386/blob/main/2026/HW/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kirchhoff's Laws and Linear Algebra: Solving Circuit Networks

## Introduction

In this homework, we'll solve resistor circuit networks using Kirchhoff's laws and linear algebra in Python. Every circuit with resistors and voltage sources can be translated into a system of linear equations — making `numpy.linalg.solve()` a powerful tool for circuit analysis. We'll practice proper documentation, docstrings, and functional programming along the way.

### Learning Outcomes
- Understand how to represent a physical system (circuit) as a system of linear equations.
- Learn to use numpy for solving linear algebra problems (`np.linalg.solve`).
- Practice writing clear and informative docstrings.
- Apply functional programming concepts by breaking the problem into small, focused functions.
- Extend code to handle more complex systems.

## Background: Kirchhoff's Laws

Kirchhoff's circuit laws are two fundamental rules that govern electrical circuits:

### Kirchhoff's Current Law (KCL)

At any node (junction) in a circuit, the sum of currents flowing in equals the sum of currents flowing out:

$$\sum_{k} I_k = 0$$

This is a direct consequence of conservation of charge.

### Kirchhoff's Voltage Law (KVL)

Around any closed loop in a circuit, the sum of all voltage drops equals zero:

$$\sum_{k} V_k = 0$$

This follows from conservation of energy. For a resistor, the voltage drop is given by Ohm's law:

$$V = I \cdot R$$

## Example Problem: A Simple Two-Loop Circuit

Consider the following circuit with two loops, three resistors, and one voltage source:

```
         R₁            R₂
  A ---/\/\/--- B ---/\/\/--- C
  |             |             |
 (+)           R₃             |
  V₀            |             |
 (-)            |             |
  |             |             |
  D ----------- E ------------
```

- $V_0 = 12$ V (voltage source between A and D)
- $R_1 = 4 \ \Omega$, $R_2 = 6 \ \Omega$, $R_3 = 3 \ \Omega$

We want to find the currents $I_1$ (through $R_1$), $I_2$ (through $R_2$), and $I_3$ (through $R_3$).

For how to draw circuits like this one check: https://github.com/Blokkendoos/AACircuit

For a figure version see end of notebook ...

### Setting Up the Equations

We define two loop currents and apply Kirchhoff's laws.

**KCL at node B:**

$$I_1 = I_2 + I_3$$

or equivalently:

$$I_1 - I_2 - I_3 = 0 \quad \text{...(1)}$$

**KVL around the left loop (A → B → E → D → A):**

$$V_0 - I_1 R_1 - I_3 R_3 = 0$$

$$I_1 R_1 + I_3 R_3 = V_0 \quad \text{...(2)}$$

**KVL around the right loop (B → C → E → B):**

$$I_2 R_2 - I_3 R_3 = 0 \quad \text{...(3)}$$

(Current $I_2$ flows through $R_2$ in the same direction as the loop, and $I_3$ flows through $R_3$ in the opposite direction for this loop.)

### Matrix Form

We can write equations (1)–(3) as a matrix equation $A \vec{I} = \vec{b}$:

$$
\begin{bmatrix}
1 & -1 & -1 \\
R_1 & 0 & R_3 \\
0 & R_2 & -R_3
\end{bmatrix}
\begin{bmatrix}
I_1 \\
I_2 \\
I_3
\end{bmatrix}
=
\begin{bmatrix}
0 \\
V_0 \\
0
\end{bmatrix}
$$

Substituting our values:

$$
\begin{bmatrix}
1 & -1 & -1 \\
4 & 0 & 3 \\
0 & 6 & -3
\end{bmatrix}
\begin{bmatrix}
I_1 \\
I_2 \\
I_3
\end{bmatrix}
=
\begin{bmatrix}
0 \\
12 \\
0
\end{bmatrix}
$$

This is exactly the kind of problem `numpy.linalg.solve()` is made for!

### Analytical Solution (for verification)

From equation (3): $I_2 = \frac{R_3}{R_2} I_3 = \frac{3}{6} I_3 = \frac{1}{2} I_3$

From equation (1): $I_1 = I_2 + I_3 = \frac{3}{2} I_3$

Substituting into equation (2):

$$\frac{3}{2} I_3 \cdot 4 + I_3 \cdot 3 = 12$$

$$6 I_3 + 3 I_3 = 12$$

$$I_3 = \frac{4}{3} \text{ A}, \quad I_2 = \frac{2}{3} \text{ A}, \quad I_1 = 2 \text{ A}$$

We can verify: $P_{\text{source}} = V_0 \cdot I_1 = 12 \cdot 2 = 24$ W, and the total power dissipated in resistors is $I_1^2 R_1 + I_2^2 R_2 + I_3^2 R_3 = 16 + \frac{8}{3} + \frac{16}{3} = 24$ W. Energy is conserved! ✓

## Coding

In [ ]:
# import libraries
import numpy as np

In [ ]:
def solve_two_loop_circuit(V0, R1, R2, R3):
    """Solve a two-loop circuit with three resistors and one voltage source.

    The circuit has the topology:
        A --R1-- B --R2-- C
        |        |        |
       V0        R3       |
        |        |        |
        D ------ E -------

    Uses Kirchhoff's current and voltage laws to set up
    a system of linear equations A @ I = b.

    Parameters
    ----------
    V0 : float
        Voltage of the source (V).
    R1 : float
        Resistance of first resistor (Ohm).
    R2 : float
        Resistance of second resistor (Ohm).
    R3 : float
        Resistance of third resistor (Ohm).

    Returns
    -------
    currents : numpy.ndarray
        Array [I1, I2, I3] of branch currents (A).
    """
    # Coefficient matrix from Kirchhoff's laws
    A = np.array([
        [1,  -1, -1 ],   # KCL at node B
        [R1,  0,  R3],   # KVL left loop
        [0,   R2, -R3]   # KVL right loop
    ], dtype=float)

    # Right-hand side vector
    b = np.array([0, V0, 0], dtype=float)

    # Solve the linear system
    currents = np.linalg.solve(A, b)
    return currents


# Example values
V0 = 12  # Volts
R1 = 4   # Ohm
R2 = 6   # Ohm
R3 = 3   # Ohm

currents = solve_two_loop_circuit(V0, R1, R2, R3)

print("Branch currents:")
print(f"I₁ = {currents[0]:.4f} A  (through R₁)")
print(f"I₂ = {currents[1]:.4f} A  (through R₂)")
print(f"I₃ = {currents[2]:.4f} A  (through R₃)")

In [ ]:
def verify_power_balance(V0, R1, R2, R3, currents):
    """Verify conservation of energy in the circuit.

    Checks that the power delivered by the voltage source equals
    the total power dissipated in all resistors.

    Parameters
    ----------
    V0 : float
        Voltage of the source (V).
    R1, R2, R3 : float
        Resistances (Ohm).
    currents : numpy.ndarray
        Array [I1, I2, I3] of branch currents (A).

    Returns
    -------
    bool
        True if power balance holds (within numerical tolerance).
    """
    I1, I2, I3 = currents
    P_source = V0 * I1
    P_dissipated = I1**2 * R1 + I2**2 * R2 + I3**2 * R3

    print(f"Power delivered by source:     {P_source:.4f} W")
    print(f"Power dissipated in resistors: {P_dissipated:.4f} W")

    return np.isclose(P_source, P_dissipated)


balanced = verify_power_balance(V0, R1, R2, R3, currents)
print(f"Energy conserved: {balanced}")

### Conclusion

We solved a simple two-loop circuit by translating Kirchhoff's laws into a linear system $A\vec{I} = \vec{b}$ and using `numpy.linalg.solve()`. The analytical solution confirms our numerical result, and the power balance serves as a unit test.

However, this code is written specifically for one circuit topology. In the homework below, you will generalize the approach to handle more complex circuits — the key advantage of a computational method over solving by hand.

## Homework (total 42 points)

Make sure that all the code you are writing has proper docstrings.

### Part 1: Bridge Circuit (32 points)

A bridge is a classic circuit used to measure unknown resistances with high precision.

```
     B ---/\/\/--- C--(-)
     |\     R₄     |
     | \           |
     R₃ \__R₅ __   R₂
     |          \  |
     |      R₁   \ |         
(+)--A ---/\/\/--- D

```

- $V_0$: voltage source between A and C
- $R_1, R_2, R_3, R_4$: bridge resistors
- $R_5$: galvanometer (bridge resistor between B and D)

#### Tasks:

1. **Set up the equations** (10 points)  
   Write a function `build_system(V0, R1, R2, R3, R4, R5)` that returns the coefficient matrix $A$ and the right-hand-side vector $\vec{b}$ for the bridge circuit using Kirchhoff's laws. Your function should return both `A` and `b`.

2. **Solve and verify** (10 points)  
   Write a function `solve_circuit(A, b)` that solves the system and returns all branch currents. Test it with $V_0 = 10$ V, $R_1 = 100 \ \Omega$, $R_2 = 200 \ \Omega$, $R_3 = 300 \ \Omega$, $R_4 = 600 \ \Omega$, $R_5 = 50 \ \Omega$.  
   Verify your result using a power balance check.

3. **Balanced bridge condition** (12 points)  
   A circuit bridge is *balanced* when no current flows through $R_5$ (the galvanometer), which happens when:
   $$\frac{R_1}{R_3} = \frac{R_2}{R_4}$$
   - Write a function that takes the four bridge resistors and checks whether the bridge is balanced.  

### Part 2: AI Creativity (10 points)

Use the following prompt in your favorite AI tool:

> I have a circuit with resistors and voltage sources that I solve using Kirchhoff's laws and linear algebra in Python. Give me a creative way to extend this circuit — for example adding capacitors, inductors, or building a practical application like a filter, sensor bridge, or measurement device. Show me how to set up and solve the equations in a Jupyter notebook. Then ask me if the circuit makes physical sense. Take my answer and adapt the model accordingly, updating the code. Repeat this until I'm confident the circuit could work in the real world.

Copy and run the final code in the cell below. Then comment on:
- How your AI model handled the physics (did it get the equations right on the first try?)
- How many iteration steps it needed to arrive at a physically realistic circuit
- Where in nature or engineering you would find the final circuit

In [ ]:
# this installs a new library
!pip install schemdraw

In [ ]:
import schemdraw
import schemdraw.elements as elm

with schemdraw.Drawing() as d:
    d += elm.SourceV().up().label('$V_0$')
    d += elm.Resistor().right().label('$R_1$')
    d.push()
    d += elm.Resistor().down().label('$R_3$')
    d += elm.Line().left()
    d.pop()
    d += elm.Resistor().right().label('$R_2$')
    d += elm.Line().down()
    d += elm.Line().left()
    d += elm.Line().left()